In [2]:
import numpy as np
import pandas as pd
import re

In [3]:
df1=pd.read_csv('anime.csv')
ratings=pd.read_csv('rating.csv')

In [4]:

result = df1.loc[df1['anime_id'] == 21]
print(result)


    anime_id       name                                              genre  \
74        21  One Piece  Action, Adventure, Comedy, Drama, Fantasy, Sho...   

   type episodes  rating  members  
74   TV  Unknown    8.58   504862  


In [5]:
df = pd.merge(df1,ratings[['anime_id','user_id']],on='anime_id')

In [6]:
result = df.loc[df['anime_id'] ==0]
print(result)

Empty DataFrame
Columns: [anime_id, name, genre, type, episodes, rating, members, user_id]
Index: []


In [7]:
def remove_extra_chars_regex(text):
  """Removes non-English letters from a string using regular expressions."""
  return re.sub(r"[^\w:]", " ", text) # Preserves hyphens for names like "Jean-Claude"

df['names'] = df['name'].apply(remove_extra_chars_regex)

In [8]:
del df['name']

In [9]:
'One Piece' in df.index


False

In [10]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

In [11]:
num_rating=df.groupby('names').count()['rating'].reset_index()
num_rating.rename(columns={'rating':'num_ratings'},inplace=True)
num_rating.head()

,names,num_ratings
0,,6
1,hack G U Returner,730
2,hack G U Trilogy,1118
3,hack G U Trilogy: Parody Mode,353
4,hack Gift,860


In [12]:
avg_rating=df.groupby('names')['rating'].mean().reset_index()
avg_rating.rename(columns={'rating':'avg_ratings'},inplace=True)
avg_rating.head()

,names,avg_ratings
0,,4.22
1,hack G U Returner,6.91
2,hack G U Trilogy,7.32
3,hack G U Trilogy: Parody Mode,6.67
4,hack Gift,6.37


In [13]:
popular_anime = pd.merge(num_rating, avg_rating, on='names')
popular_anime.head()

,names,num_ratings,avg_ratings
0,,6,4.22
1,hack G U Returner,730,6.91
2,hack G U Trilogy,1118,7.32
3,hack G U Trilogy: Parody Mode,353,6.67
4,hack Gift,860,6.37


In [14]:
popular_anime[popular_anime['num_ratings']>=100].sort_values('avg_ratings',ascending=False)

,names,num_ratings,avg_ratings
4949,Kimi no Na wa,2199,9.37
2599,Fullmetal Alchemist: Brotherhood,24574,9.26
3046,Gintama,1386,9.25
9354,Steins Gate,19283,9.17
3047,Gintama 039,3673,9.16
...,...,...,...
7965,Pupa,2677,3.82
10519,Vampire Holmes,162,3.63
8957,Shitcom,231,2.93
3372,Hametsu no Mars,1131,2.37


In [15]:
popular_anime=popular_anime.merge(df,on='names').drop_duplicates('names')[['names','genre','type','episodes','num_ratings','avg_ratings']]

In [16]:
popular_anime.head()

,names,genre,type,episodes,num_ratings,avg_ratings
0,,"Dementia, Music",ONA,1,6,4.22
6,hack G U Returner,"Adventure, Drama, Fantasy, Game, Magic, Sci-Fi",OVA,1,730,6.91
736,hack G U Trilogy,"Action, Fantasy, Game, Sci-Fi",Movie,1,1118,7.32
1854,hack G U Trilogy: Parody Mode,"Comedy, Fantasy, Game, Parody, Sci-Fi",Special,1,353,6.67
2207,hack Gift,"Comedy, Fantasy, Game",OVA,1,860,6.37


In [17]:
x=df.groupby('user_id').count()['rating']>100
weeb=x[x].index

In [18]:
ani=df[df['user_id'].isin(weeb)]

In [19]:
y=ani.groupby('names').count()['rating']>=50
famous_anime=y[y].index

In [20]:
final_ratings=ani[ani['names'].isin(famous_anime)]

In [21]:
final_ratings

,anime_id,genre,type,episodes,rating,members,user_id,names
0,32281,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,Kimi no Na wa
1,32281,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,Kimi no Na wa
2,32281,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,Kimi no Na wa
3,32281,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,Kimi no Na wa
6,32281,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,398,Kimi no Na wa
...,...,...,...,...,...,...,...,...
7813155,1639,"Hentai, Yaoi",OVA,1,5.13,57355,73328,Boku no Pico
7813156,1639,"Hentai, Yaoi",OVA,1,5.13,57355,73329,Boku no Pico
7813158,1639,"Hentai, Yaoi",OVA,1,5.13,57355,73350,Boku no Pico
7813159,1639,"Hentai, Yaoi",OVA,1,5.13,57355,73389,Boku no Pico


In [22]:
pt=final_ratings.pivot_table(index='names',columns='user_id',values='rating')

In [23]:
pt.fillna(0,inplace=True)

In [24]:
'One Piece' in pt.index


False

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
similarity_scores=cosine_similarity(pt)

In [27]:
similarity_scores.shape

(5531, 5531)

In [28]:
def recommend(anime_name):
    if anime_name in pt.index:
        index = np.where(pt.index == anime_name)[0][0]
        similar_items = sorted(list(enumerate(similarity_scores[index])), key=lambda x: x[1], reverse=True)[1:6]
        for i in similar_items:
            print(pt.index[i[0]])
    else:
        print("Anime not found in the dataset.")


In [29]:
recommend('Death Note')

Code Geass: Hangyaku no Lelouch
Code Geass: Hangyaku no Lelouch R2
Elfen Lied
Shingeki no Kyojin
Fullmetal Alchemist: Brotherhood
